In [2]:
import pandas as pd
import numpy as np
import pprint
from ml_utils import Utils

from nltk.stem.snowball import SnowballStemmer
from sklearn import tree
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.grid_search import GridSearchCV

from pylab import figure, axes, title, savefig
import matplotlib.pyplot as plt

MIN_VAL = 1.0
MAX_VAL = 3.0

**Note about files being read:** These inputs are produced by running the `str_stem` function. They are essentially the files provided in the competition, but the strings have been pre-processed to eliminate irrelevant characters and other interferences, as well as to stem the strings, i.e., minimize the differences from inflexions found in the language (e.g., "cat" and "cats" are stemmed to the same word to maximize matches once the numerical features are calculated).

In [3]:
df_train = pd.read_csv("../input/_final_train.csv", encoding="ISO-8859-1")
df_test = pd.read_csv('../input/_final_test.csv', encoding="ISO-8859-1")
df_attr = pd.read_csv('../input/attributes.csv')
df_prod_desc = pd.read_csv('../input/_final_desc.csv', index_col="product_uid")
ytrain = df_train["relevance"]

In [4]:
# Make sure it worked
print df_prod_desc.head()

             Unnamed: 0                                product_description
product_uid                                                               
100001                0  not on do angl make joint stronger they also p...
100002                1  behr premium textur deckov is an innov solid c...
100003                2  classic architectur meet contemporari design i...
100004                3  the grape solar 265watt. polycrystallin pv sol...
100005                4  updat your bathroom with the delta vero singl ...


## Utilities

In [5]:
def print_scores(clf, verbose=False):
    """
    Prints the best scores and best parameters found in cross-validation.
    Takes:
    - clf, a classifier that has been cross-validated by grid search.
    Returns:
    """
    pp = pprint.PrettyPrinter(indent=4)
    print "Best score and parameters:"
    pp.pprint(clf.best_score_)
    pp.pprint(clf.best_params_)
    
    if verbose:
        print "~~~Verbose output~~~"
        print "All scores"
        pp.pprint(clf.grid_scores_)
    
def fmean_squared_error(ground_truth, predictions):
    """
    Returns the difference between the actual value and the predictions
    output by a classifier
    Takes:
    - ground_truth, the actual value Y of the inputs X being predicted.
    - predictions, the predicted value Yhat of the inputs X.
    Returns:
    - 
    """
    fmean_squared_error_ = mean_squared_error(ground_truth, predictions)**0.5
    return fmean_squared_error_

def is_prediction_in_range(yhat, min_val, max_val, verbose=False):
    """Returns whether the values of a prediction are in the expected range"""
    return min(yhat) >= min_val and max(yhat) <= max_val

def export_data(yhat, is_test, predictor_name):
        output_folder = "../output/"

        if not is_test:
            # Most common words in data with the worst errors
            df_word_cloud = pd.DataFrame(0, index=np.arange(len(df_train)),\
                                    columns=["product_title", "search_term",\
                                         "Y", "Yhat", "diff"])

            # Exports Y, Yhat and difference between them.
            # Can be helpful for tuning or seeing patterns
            df_num = pd.DataFrame(0, index=np.arange(len(df_train)),\
                                    columns=["Y", "Yhat", "diff"])

            # Word cloud
            df_word_cloud["product_title"] = df_train["product_title"]
            df_word_cloud["search_term"] = df_train["search_term"]
            df_word_cloud["Y"] = df_train["relevance"]
            df_word_cloud["Yhat"] = yhat
            df_word_cloud["diff"] = (df_word_cloud["Y"] - df_word_cloud["Yhat"]) ** 2

            # Diff
            df_num["Y"] = df_train["relevance"]
            df_num["Yhat"] = yhat
            df_num["diff"] = df_word_cloud["diff"]

            output_file_words = output_folder + predictor_name + "_word_cloud.csv"
            output_file_num = output_folder + predictor_name + "_num.csv"

            df_word_cloud.to_csv(output_file_words, encoding="utf-8")
            df_num.to_csv(output_file_num, encoding="utf-8")

        else:
            # Exports Yhat(testing data). This is the submission file
            # df_result = pd.DataFrame(0, index=np.arange(len(df_test)),
              #                      columns=["id", "relevance"])

            df_result = pd.DataFrame(yhat, columns=["relevance"],\
                                    index=df_test["id"])

            output_file_yhat = output_folder + predictor_name + "_yhat.csv"

            df_result.to_csv(output_file_yhat, encoding="utf-8")


RMSE = make_scorer(fmean_squared_error, greater_is_better=False)

## Create numerical features
### Simple feature extraction

In [8]:
train_num_feat = pd.DataFrame(0, index=np.arange(len(df_train)), columns=["search_in_title",\
                                     "search_in_desc"])
test_num_feat = pd.DataFrame(0, index=np.arange(len(df_test)), columns=["search_in_title",\
                                     "search_in_desc"])

# Count the training data occurrences
for i in range(0, df_train.shape[0]):
    search = df_train["search_term"][i]  # Search term used by user
    title = df_train["product_title"][i] # Product Title
    product_id = df_train.iloc[[i]]["product_uid"][i]  # Get product_uid of current index in df_train
    description = df_prod_desc.loc[product_id]["product_description"] # Description of product id
    
    for word in search.split():

        if word in title:
            train_num_feat["search_in_title"][i] += 1
            
        if word in description:
            train_num_feat["search_in_desc"][i] += 1
            
# Count the testing data occurrences
for i in range(0, df_test.shape[0]):
    search = df_test["search_term"][i]  # Search term used by user
    title = df_test["product_title"][i] # Product Title
    product_id = df_test.iloc[[i]]["product_uid"][i]  # Get product_uid of current index in df_train
    description = df_prod_desc.loc[product_id]["product_description"] # Description of product id

    for word in search.split():
        
        if word in title:
            test_num_feat["search_in_title"][i] += 1
        
        if word in description:
            test_num_feat["search_in_desc"][i] += 1

#### Export them

In [9]:
print train_num_feat.head(10)
#train_num_feat.to_csv("../input/simple_train_num_feat.csv")
#test_num_feat.to_csv("../input/simple_test_num_feat.csv")

   search_in_title  search_in_desc
0                1               1
1                1               1
2                1               1
3                1               1
4                3               2
5                1               2
6                2               2
7                1               1
8                2               2
9                2               2


### Or import them 
#### Simple

In [6]:
train_num_feat = pd.read_csv("../input/simple_train_num_feat.csv", usecols=[1,2])
test_num_feat = pd.read_csv("../input/simple_test_num_feat.csv", usecols=[1,2])

train_num_feat.head()

,search_in_title,search_in_desc
0,1,1
1,1,1
2,1,2
3,1,1
4,3,3


#### Or more complex

In [7]:
train_num_feat = pd.read_csv("../input/more_features/lab_x_train.csv")
test_num_feat = pd.read_csv("../input/more_features/lab_x_test.csv")

train_num_feat.head()

,query_len,title_len,desc_len,brand_len,query_in_title,query_in_desc,query_last_word_in_title,query_last_word_in_desc,word_in_title,word_in_desc,ratio_title,ratio_description,word_in_brand,ratio_brand,brand_feature,search_term_feature
0,2,6,135,3,0,0,0,0,1,1,0.500,0.500,0,0.00,1000,12
1,2,6,135,3,0,0,0,0,1,1,0.500,0.500,0,0.00,1000,9
2,2,12,169,4,0,0,0,0,1,1,0.500,0.500,1,0.25,1000,9
3,3,14,109,1,0,0,0,0,1,1,0.333,0.333,0,0.00,1010,16
4,3,14,109,1,1,0,1,1,3,3,1.000,1.000,0,0.00,1010,18


## Fit the model and predict both testing and training values
### Hand-tuning parameters
At first, I played with various parameters to the predictor and used it to predict training data. If the decision tree is not stopped early enough, the model will clearly overfit, resulting in a training MSE of ~0.001 with the complex feature set. I thought it would be an interesting experiment to try to limit the overfitting by matching the _training_ errors of the predictors that we had created with the best public _testing_ error so far (training MSE ~0.22). Interestingly enough, so far this hand-tuned version of the decision tree is the version that has produced the best public testing score out of all decision tree regressors we have tried, scoring better than the cross-validated model that should select better values for the parameters (see next section). This is, of course, not necessarily representative of the final performance of the models, since before the deadline the submissions are only scored on a portion of the testing data, but I thought it was interesting enough to mention.

In [8]:
clf = tree.DecisionTreeRegressor(min_samples_split=3, max_depth=6)
# Train
clf.fit(train_num_feat, df_train["relevance"])

# Predict training and testing data
yhat_train_val = clf.predict(train_num_feat)
yhat_test_val = clf.predict(test_num_feat)

# Make sure results are within boundaries
print "Training Yhat in range: " + str(is_prediction_in_range(yhat_train_val, MIN_VAL, MAX_VAL))
print "Testing Yhat in range: " + str(is_prediction_in_range(yhat_test_val, MIN_VAL, MAX_VAL))
# MSE
err = (df_train["relevance"] - yhat_train_val) ** 2
mse_train = sum(err) / df_train.shape[0]
print "Training MSE: " + str(mse_train)

export_data(yhat_test_val, True, "decision_tree_1_depth_lim")

Training Yhat in range: True
Testing Yhat in range: True
Training MSE: 0.228578828314


### Cross-validation

In order to cross validate, I will use `GridSearchCV`, a built-in scikit module that tries the different combinations of parameters that it is passed on a predictor, and stores the best parameters.

In [ ]:
# Create the predictor
tree_multi = tree.DecisionTreeRegressor()

# Create parameter grid: Combinations of all of these parameters will be tried
# with the predictor
splitter = ["best", "random"]
max_features = np.linspace(1, 16, 5).astype(int)
max_depth = np.linspace(2, 10, 5)
min_samples_split = np.linspace(1, 5, 5)
min_samples_leaf = np.linspace(1, 5, 5)
presort = [True, False]

param_grid = dict(splitter=splitter, max_features=max_features,\
                  max_depth=max_depth,\
                  min_samples_split=min_samples_split,\
                  min_samples_leaf=min_samples_leaf,\
                  presort=presort
                 )

# Cross validate!
reg_multi = GridSearchCV(tree_multi, param_grid=param_grid, scoring=RMSE)
# Fit the final model on the training data
reg_multi.fit(train_num_feat, ytrain)

In [11]:
yhat_multi = reg_multi.predict(test_num_feat)

The parameters obtained from cross-validation follow. Run with `verbose=True` to see looong output.

In [12]:
print_scores(reg_multi)

Best score and parameters:
-0.48526348210881798
{   'max_depth': 8.0,
    'max_features': 12,
    'min_samples_leaf': 4.0,
    'min_samples_split': 2.0,
    'presort': True,
    'splitter': 'best'}


### Export output and  some data for plotting

In [114]:
#export_data(yhat_train_val, "decision_tree_depth_lim")
#my_new = Utils()
#my_utils.export_data(yhat_multi, False, "decision_tree_multi")

export_train_data(yhat_multi, True, "decision_tree_multi")

    relevance
id           
1    2.051705
4    2.070831
5    2.070831
6    2.806887
7    2.235381


In [53]:
my_utils = Utils()

In [13]:
from sklearn.learning_curve import learning_curve
#from sklearn.svm import SVC

train_sizes, train_scores, valid_scores = learning_curve(reg_multi, train_num_feat, ytrain, train_sizes=[50, 80, 110], cv=5)            
print train_sizes
print train_scores
print valid_scores

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/grid_search.py:418: ChangedBehaviorWarning: The long-standing behavior to use the estimator's score function in GridSearchCV.score has changed. The scoring parameter is now used.
  ChangedBehaviorWarning)


[ 50  80 110]
[[-0.42462729 -0.4012964  -0.353271   -0.38224442 -0.32380649]
 [-0.40840869 -0.47363578 -0.46290047 -0.43957141 -0.48610995]
 [-0.43593477 -0.44740311 -0.47697519 -0.46180781 -0.46838922]]
[[-0.53362399 -0.58833978 -0.60228123 -0.59322528 -0.64438801]
 [-0.5335377  -0.51990067 -0.51768832 -0.53989008 -0.53184605]
 [-0.5180814  -0.52918906 -0.50461774 -0.52132608 -0.59077028]]


In [1]:
print train_sizes
print train_scores
print valid_scores

print "~~~~"
print train_scores_mean
print "~~~~"

figure()
plt.figure()
plt.title("Cross-validation Error")

# In order to plot, need to get the average of all cross validation iterations
train_scores_mean = np.mean(train_scores, axis=1)
#train_scores_std = np.std(train_scores, axis=1)
valid_scores_mean = np.mean(valid_scores, axis=1)
#test_scores_std = np.std(valid_scores, axis=1)

plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")

plt.plot(train_sizes, valid_scores_mean, 'o-', color="g",
             label="Cross-validation score")


savefig('foo.png')

NameError: name 'train_sizes' is not defined

In [20]:
print df_test.loc[0, "product_title"]
print test_num_feat.iloc[[0]]

 simpson strong tie 12 gaug angl
   query_len  title_len  desc_len  brand_len  query_in_title  query_in_desc  \
0          2          6       135          3               0              0   

   query_last_word_in_title  query_last_word_in_desc  word_in_title  \
0                         0                        0              0   

   word_in_desc  ratio_title  ratio_description  word_in_brand  ratio_brand  \
0             0            0                  0              0            0   

   brand_feature  search_term_feature  
0           1000                   14  


In [9]:
df_test.loc[[df_test.shape[0] - 1]]

,Unnamed: 0,id,product_uid,product_title,search_term
166692,166692,240760,224428,bosch 4in. bi metal hole saw,4in. hole saw


In [49]:
df_test.loc[:, "product_title"].iloc[[1]]

1    Simpson Strong-Tie 12-Gauge Angle
Name: product_title, dtype: object